In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from eval import custom_metric
from preprocess import data
from pygam import s, LinearGAM, te
from pykalman import KalmanFilter

In [ ]:
def MAE_loss_function (y_hat, y):
    return mean_absolute_error(y_hat, y)


In [ ]:
def custom_metric(y_hat,y, country:str):
    return custom_metric(y_hat, y, country)[0]

In [ ]:
## partie à modifier
def loss_function(y_hat,y) :
    return custom_metric(y_hat, y, "Germany")

In [3]:
time_series_austria=data['y_austria']
time_series_germany=data['y_germany']
features_austria=data['x_austria']
features_germany=data['x_germany']
regression_germany = time_series_germany.tolist()
regression_austria = time_series_austria.tolist()

In [4]:
features_austria.drop(columns=['DATE_FROM'], inplace=True)
features_germany.drop(columns=['DATE_FROM'], inplace=True)

In [5]:

# Générer les données X et y
X_austria = []
X_austria_no_features = []
y_austria = []
X_germany = []
X_germany_no_features = []
y_germany = []
X_austria_no_features_no_time = []
X_germany_no_features_no_time = []
X_austria_no_time = []
X_germany_no_time = []

for i in range(7, len(regression_austria)):
    X_austria.append(np.concatenate((regression_austria[i-7:i], features_austria.iloc[i].values)))
    # X_austria_no_features_no_time.append([regression_austria[i-1:i][0], features_austria.iloc[i].values[-1]])
    X_austria_no_features_no_time.append(regression_austria[i-1:i])
    X_austria_no_time.append(np.concatenate((regression_austria[i-1:i], features_austria.iloc[i].values)))
    X_austria_no_features.append(regression_austria[i-7:i])
    y_austria.append(regression_austria[i])

for i in range(7, len(regression_germany)):
    X_germany.append(np.concatenate((regression_germany[i-7:i], features_germany.iloc[i].values)))
    X_germany_no_features_no_time.append(regression_germany[i-1:i])
    X_germany_no_time.append(np.concatenate((regression_germany[i-1:i], features_germany.iloc[i].values)))
    X_germany_no_features.append(regression_germany[i-7:i])
    y_germany.append(regression_germany[i])

print(X_austria_no_features_no_time)
X_austria = np.array(X_austria)
X_austria_no_features = np.array(X_austria_no_features)
y_austria = np.array(y_austria)
X_germany = np.array(X_germany)
X_germany_no_features = np.array(X_germany_no_features)
y_germany = np.array(y_germany)
X_austria_no_features_no_time = np.array(X_austria_no_features_no_time)
X_germany_no_features_no_time = np.array(X_germany_no_features_no_time)
X_austria_no_time = np.array(X_austria_no_time)
X_germany_no_time = np.array(X_germany_no_time)

print("X shape:", X_austria.shape)
print("X no features shape:", X_austria_no_features.shape)
print("y shape:", y_austria.shape)

[[4.04], [4.09], [3.67], [3.41], [3.46], [3.2], [3.38], [3.08], [3.0], [2.98], [2.86], [2.76], [2.57], [2.48], [2.44], [4.16], [3.96], [3.5], [3.5], [3.08], [3.48], [3.23], [3.98], [3.66], [3.51], [3.35], [3.05], [3.11], [4.53], [3.29], [3.11], [2.96], [2.9], [2.88], [2.8], [2.7], [2.8], [2.51], [2.34], [2.3], [2.1], [2.13], [1.91], [2.57], [2.83], [2.49], [2.38], [2.13], [2.35], [2.61], [2.66], [2.64], [4.97], [3.49], [3.88], [3.05], [2.97], [2.94], [3.1], [3.11], [3.06], [2.8], [2.89], [3.13], [3.92], [3.86], [3.48], [3.2], [2.95], [2.83], [3.53], [3.29], [3.03], [2.8], [2.72], [2.86], [2.75], [5.93], [4.57], [4.27], [3.98], [3.66], [5.49], [7.88], [8.89], [7.47], [6.6], [6.18], [5.44], [5.01], [4.71], [4.58], [4.75], [4.74], [4.48], [4.81], [5.13], [6.9], [6.59], [7.46], [6.23], [6.56], [11.2], [12.18], [9.82], [8.36], [8.91], [9.98], [8.66], [8.15], [7.46], [7.81], [7.23], [7.12], [6.86], [7.46], [6.83], [6.89], [7.07], [6.77], [6.58], [6.41], [6.08], [6.55], [6.0], [5.81], [5.78],

In [7]:
def give_y_pred(X,y, alpha):
    assert len(X) == len(y)
    y_preds = []
    for i in range(len(X)-100):
        X_train = X[:i+100]
        y_train = y[:i+100]
        ridge = Ridge(alpha=alpha)
        ridge.fit(X_train, y_train)
        y_pred = ridge.predict([X[i+100]])
        y_preds.append(y_pred)
        
    return y_preds

In [9]:
from sklearn.model_selection import GridSearchCV
from scipy.optimize import minimize
from sklearn.linear_model import Ridge

def get_aggregator_prediction(X,y,i):
    y_train = y[i:i+100]
    X_train = X[i:i+100]
    ridge = Ridge(alpha=60)
    ridge.fit([[X_train[f,6]] for f in range(100)], y_train)
    y_pred_linear = ridge.predict([[X_train[f,6]] for f in range(100)])
    XGB = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    XGB.fit(X_train, y_train)
    RandomForest = RandomForestRegressor(n_estimators=60, random_state=42)
    RandomForest.fit(X_train, y_train)
    gam = LinearGAM(s(6, lam=100) + s(-1, lam=10))
    y_pred_gam_temp = gam.fit(X_train, y_train).predict(X_train)
    kf = KalmanFilter(initial_state_mean=y_pred_gam_temp[0], n_dim_obs=1)
    state_means, _ = kf.smooth(y_train)
    y_pred_gam = state_means.flatten()
    y_pred_rf = RandomForest.predict(X_train)
    y_pred_xgb = XGB.predict(X_train)
    # y_preds = np.array([y_pred_linear, y_pred_xgb, y_pred_rf, np.zeros(100)])
    y_preds = np.array([y_pred_linear, y_pred_xgb, y_pred_rf, y_pred_gam, np.zeros(100)])
    y_next_pred_gam_temp = gam.predict(X[i+100:i+101])
    kf = KalmanFilter(initial_state_mean=y_next_pred_gam_temp[0], n_dim_obs=1)
    state_means_next_pred, _ = kf.smooth(y_next_pred_gam_temp)
    y_next_pred_gam = state_means_next_pred.flatten()
    def objective(weights):
        y_pred_combined = np.dot(weights, y_preds)
        return loss_function(y_pred_combined, y_train)
        

    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1)] * y_preds.shape[0]
    initial_weights = [1.0 / y_preds.shape[0]] * y_preds.shape[0]

    result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    best_weights = result.x
    print(result.x)

    y_next_preds = np.array([ridge.predict([[X[i+100,6]]]), XGB.predict(X[i+100:i+101]),RandomForest.predict(X[i+100:i+101]),y_next_pred_gam,[0]])
    y_pred_combined = np.dot(best_weights, y_next_preds)
    return y_pred_combined
    

    

In [11]:
def get_aggregator_score(X,y):
    y_preds = []
    for i in range(0, len(y)-100):
        y_preds.append(get_aggregator_prediction(X,y,i))
    y_preds = np.concatenate(y_preds)
    return loss_function(y_preds, y[100:])

In [12]:
get_aggregator_score(X_germany,y_germany)

[0.00000000e+00 1.00000000e+00 1.38777878e-17 8.32667268e-17
 0.00000000e+00]
[0. 1. 0. 0. 0.]
[5.77315973e-15 1.00000000e+00 2.30371278e-15 4.60742555e-15
 1.13554999e-14]
[0. 1. 0. 0. 0.]
[5.74540415e-15 1.00000000e+00 6.99440506e-15 9.71445147e-16
 2.44249065e-15]
[0.00000000e+00 1.00000000e+00 4.92985945e-15 0.00000000e+00
 0.00000000e+00]
[5.08683268e-15 1.00000000e+00 0.00000000e+00 1.08246745e-15
 7.57637214e-16]
[0.00000000e+00 1.00000000e+00 7.14453258e-15 0.00000000e+00
 0.00000000e+00]
[1.50282637e-15 1.00000000e+00 0.00000000e+00 1.38777878e-16
 9.66101920e-16]
[5.50269211e-15 1.00000000e+00 0.00000000e+00 6.05608847e-16
 1.67574288e-15]
[1.10799179e-15 1.00000000e+00 7.10542736e-15 2.49800181e-16
 1.90237464e-16]
[0. 1. 0. 0. 0.]
[7.33085671e-16 1.00000000e+00 0.00000000e+00 2.22044605e-16
 0.00000000e+00]
[0. 1. 0. 0. 0.]
[0.00000000e+00 1.00000000e+00 0.00000000e+00 2.89901473e-15
 0.00000000e+00]
[1.03560464e-14 1.00000000e+00 3.62501522e-15 6.11448848e-16
 2.21871133e-

3.1140694050433892